# Redefine the same model adding new methods

In [ ]:
import pickle
import torch
import torch.nn as nn
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

class Classifier(nn.Module):
    def __init__(self,
                 ngpu,
                 input_size: int = 768,
                 num_classes: int = 2,
                 decoder: dict = None):
        super(Classifier, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(nn.Conv1d(input_size, 256, kernel_size=3, stride=1, padding=1),
                                  nn.ReLU(),
                                  nn.MaxPool1d(kernel_size=2, stride=2),
                                  nn.Conv1d(256, 128, kernel_size=3, stride=1, padding=1),
                                  nn.ReLU(),
                                  nn.MaxPool1d(kernel_size=2, stride=2),
                                  nn.Conv1d(128, 64, kernel_size=3, stride=1, padding=1),
                                  nn.ReLU(),
                                  nn.MaxPool1d(kernel_size=2, stride=2),
                                  nn.Conv1d(64, 32, kernel_size=3, stride=1, padding=1),
                                  nn.ReLU(),
                                  nn.Flatten(),
                                  nn.Linear(384 , 128),
                                  nn.ReLU(),
                                  nn.Linear(128, num_classes))
        self.decoder = decoder
    def forward(self, inp):
        x = self.main(inp)
        return x

    def predict(self, inp):
        with torch.no_grad():
            x = self.forward(inp)
            return x
    
    def predict_proba(self, inp):
        with torch.no_grad():
            x = self.forward(inp)
            return torch.softmax(x, dim=1)

    def predict_classes(self, inp):
        with torch.no_grad():
            x = self.forward(inp)
            return self.decoder[int(torch.argmax(x, dim=1))]


# Use the TextProcessor 

In [ ]:
from text_process import TextProcessor
# Make sure that you use the same max_length that you used when you trained the model
text_processor = TextProcessor(max_length=100)

with open('text_decoder.pkl', 'rb') as image_decoder:
    image_decoder = pickle.load(image_decoder)


model = Classifier(ngpu=1, num_classes=13, decoder=image_decoder)
model.load_state_dict(torch.load('model_bert.pt', map_location='cpu'))
model.eval()
model.to('cpu')